# Match - Match emissions with suffix `ion`

There are many flows which are exactly the same, but where ecoinvent has the suffix `, ion` which is missing in SimaPro, e.g.

* Arsenic
* Arsenic, ion

// These are all natural resources; `('Resource', 'in ground')` in SimaPro, `('natural resource', 'in ground')` in ecoinvent.

In [1]:
import pandas as pd
from pathlib import Path
from notebook_utils import *

In [2]:
input_data_dir = (Path.cwd().parent / "Mapping" / "Input" / "Flowlists").resolve()

In [3]:
sp = pd.read_csv(input_data_dir / 'SimaProv9.4.csv')

Add additional column to `sp` with `in ground`

In [4]:
sp['plus_ion'] = sp.Flowable.apply(lambda x: x + ", ion")

Filter to only consider natural resources:

In [5]:
ei = pd.read_csv(input_data_dir / 'ecoinventEFv3.7.csv')

In [6]:
for c in ("air", "water", "soil", "resources"):
    sp_filtered = expand_simapro_context(sp, c)
    sp_filtered = add_ecoinvent_context_column(sp_filtered, label="EcoinventContext", kind=c)
    df = sp_filtered.merge(
        ei, 
        how="inner", 
        left_on=["plus_ion", "EcoinventContext"], 
        right_on=["Flowable", "Context"]
    )
    df["ConversionFactor"] = 1.0    
    finish_notebook(
        df=df, 
        author="Chris Mutel",
        notebook_name="Match - Match emissions with suffix ion",
        filename=f'suffix-ion-{c}'
    )